In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from fuzzywuzzy import fuzz
import string

C:\Users\devquinn\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Data Preprocessing

In [2]:
path = 'C:/Users/devquinn/OneDrive - Deloitte (O365D)/Desktop/atom-dc-coa/'
write_path = 'C:/Users/devquinn/OneDrive - Deloitte (O365D)/Desktop/atom-dc-coa/dc-atom-coa/project/data/'

## Company: Fanatics

In [3]:
file = 'Fanatics/GL_Account_Mapping_V5.11.xlsx'
df = pd.read_excel(open(path+file, 'rb'), sheet_name="GL Account Mapping", dtype={'Source Account #': str, 'Target Account #': str}, header=0)
len(df)

6242

In [4]:
#filter for BS accounts
df = df.query(f'`Account Type` == "Balancesheet"')
print(len(df))
df.head(3)

1375


,Account Type,Source Level 1,Source Level 2,Source Level 3,Source Management P&L,Source Account #,Source Account Description,Target Level 1,Target Level 2,Target Level 3,Target Management PnL,Target Account #,Target Account Description,Unnamed: 13
0,Balancesheet,Assets,Current Assets,Cash and Cash Equivalents,Cash and Cash Equivalents,100110,Petty Cash,Assets,Current Assets,Cash and Cash Equivalents,Cash and Cash Equivalents,100001,Petty Cash,NaN
1,Balancesheet,Assets,Current Assets,Cash and Cash Equivalents,Cash and Cash Equivalents,100111,Petty Cash - Dreams,Assets,Current Assets,Cash and Cash Equivalents,Cash and Cash Equivalents,100001,Petty Cash,NaN
2,Balancesheet,Assets,Current Assets,Cash and Cash Equivalents,Cash and Cash Equivalents,100120,Store Fund - Avenues,Assets,Current Assets,Cash and Cash Equivalents,Cash and Cash Equivalents,100003,Store Fund - Avenues,NaN


In [5]:
cols = [5,1,2,3,6,11,7,8,9,12]
df_fan = df[df.columns[cols]].dropna()
#df_fan = df_fan.rename(columns = {"Source Level 1": "S L1", "Source Level 2": "S L2", "Source Level 3": "S L3", "Source Account Description": "Source Description",
#                                  "Target Level 1": "T L1", "Target Level 2": "T L2", "Target Level 3": "T L3", "Target Account Description": "Target Description"})


df_fan.insert(0, 'Division', 'Manufacturing')
df_fan.insert(1, 'Industry', 'Misc Apparel and Accessories')


df_fan.head(3)

,Division,Industry,Source Account #,Source Level 1,Source Level 2,Source Level 3,Source Account Description,Target Account #,Target Level 1,Target Level 2,Target Level 3,Target Account Description
0,Manufacturing,Misc Apparel and Accessories,100110,Assets,Current Assets,Cash and Cash Equivalents,Petty Cash,100001,Assets,Current Assets,Cash and Cash Equivalents,Petty Cash
1,Manufacturing,Misc Apparel and Accessories,100111,Assets,Current Assets,Cash and Cash Equivalents,Petty Cash - Dreams,100001,Assets,Current Assets,Cash and Cash Equivalents,Petty Cash
2,Manufacturing,Misc Apparel and Accessories,100120,Assets,Current Assets,Cash and Cash Equivalents,Store Fund - Avenues,100003,Assets,Current Assets,Cash and Cash Equivalents,Store Fund - Avenues


In [6]:
#shuffle data
df_fan = df_fan.sample(frac=1)
df_fan.to_csv(write_path+'CompanyA.csv', index=False)

## Company: Mars

In [7]:
file = "Mars/Mars Data.xlsx"
df_map = pd.read_excel(open(path+file, 'rb'), sheet_name="Mapping", dtype = str, header=0).query(f'ERP == "Atlas"')
df_target = pd.read_excel(open(path+file, 'rb'), sheet_name="Target COA", dtype = str, header=0)
df_source = pd.read_excel(open(path+file, 'rb'), sheet_name="Source - Atlas ERP", dtype = str, header=0)
#df_source2 = pd.read_excel(open(path+file, 'rb'), sheet_name="Source - P12 ERP", dtype = str, header=0)

In [8]:
df_map.head(1)

,ERP,Source GL Account,Source GL Account Description,Target GL Account,Target GL Account Description
0,Atlas,110000,Sales at List Price (Auto),400000,Gross Sales Products - 3rd Party Domestic - Auto


In [9]:
df_mars = pd.DataFrame()
sources_not_found = []
targets_not_found = []

for i in range(len(df_map)):

    source = df_map.loc[i, 'Source GL Account']
    source_description = df_map.loc[i, 'Source GL Account Description']
    source_row = df_source.where(df_source['Account'] == source).dropna(how='all')
    if len(source_row) == 0:
        sources_not_found.append(source)
        continue
    source_hierarchy = source_row.iloc[0][1:].tolist()

    target = df_map.loc[i, 'Target GL Account']
    target_description = df_map.loc[i, 'Target GL Account Description']
    target_row = df_target.where(df_target['Account'] == target).dropna(how='all')
    if len(target_row) == 0:
        targets_not_found.append(target)
        #continue
        target_hierarchy = [np.nan, np.nan, np.nan, np.nan]
    else:
        target_hierarchy = target_row.iloc[0][:4].tolist()

    df_mars = df_mars.append({'Source Account #': source, 'Source Level 1': source_hierarchy[0], 
                            'Source Level 2': source_hierarchy[1], 'Source Level 3': source_hierarchy[2],
                            'Source Level 4': source_hierarchy[3], 'Source Level 5': source_hierarchy[4],
                            'Source Level 6': source_hierarchy[5], 'Source Level 7': source_hierarchy[6],
                            'Source Level 8': source_hierarchy[7], 'Source Level 9': source_hierarchy[8],
                            'Source Level 10': source_hierarchy[9], 'Source Account Description': source_description,

                            'Target Account #': target, 'Target Level 1': target_hierarchy[0], 
                            'Target Level 2': target_hierarchy[1], 'Target Level 3': target_hierarchy[2],
                            'Target Level 4': target_hierarchy[3], 'Target Account Description': target_description,
                            }, ignore_index=True)

if len(sources_not_found) > 0 or len(targets_not_found) > 0:
    print(f'Error: some sources/targets from mapping sheet not found in hierarchy sheet. These entries have been dropped')

df_mars.head(3)

Error: some sources/targets from mapping sheet not found in hierarchy sheet. These entries have been dropped


,Source Account #,Source Level 1,Source Level 2,Source Level 3,Source Level 4,Source Level 5,Source Level 6,Source Level 7,Source Level 8,Source Level 9,Source Level 10,Source Account Description,Target Account #,Target Level 1,Target Level 2,Target Level 3,Target Level 4,Target Account Description
0,110000,M003 Mars Financial Statement V2 (Full Trial B...,Profit and Loss,Operating Profit,6. Total Net Sales,3. Net Sales - third party domestic,Gross Sales - Domestic,NaN,NaN,NaN,NaN,Sales at List Price (Auto),400000,Revenue & Trade Expenditures,Revenue - 3rd Party,Revenue - Products - 3rd Party,Revenue - Products - 3rd Party Domestic,Gross Sales Products - 3rd Party Domestic - Auto
1,110005,M003 Mars Financial Statement V2 (Full Trial B...,Profit and Loss,Operating Profit,6. Total Net Sales,3. Net Sales - third party domestic,Gross Sales - Domestic,NaN,NaN,NaN,NaN,Sales at List Price - BIFG (Auto),400000,Revenue & Trade Expenditures,Revenue - 3rd Party,Revenue - Products - 3rd Party,Revenue - Products - 3rd Party Domestic,Gross Sales Products - 3rd Party Domestic - Auto
2,110010,M003 Mars Financial Statement V2 (Full Trial B...,Profit and Loss,Operating Profit,6. Total Net Sales,3. Net Sales - third party domestic,Gross Sales - Domestic,NaN,NaN,NaN,NaN,Sales at List Price Adjustment - Manual,400000,Revenue & Trade Expenditures,Revenue - 3rd Party,Revenue - Products - 3rd Party,Revenue - Products - 3rd Party Domestic,Gross Sales Products - 3rd Party Domestic - Auto


In [10]:
print(len(sources_not_found), len(targets_not_found))

213 6


In [11]:
sources_dfmap = set(df_map['Source GL Account'])
targets_dfmap = set(df_map['Target GL Account'])
sources_dfsource = set(df_source['Account'])
targets_dftarget = set(df_target['Account'])

#print(len(sources_dfmap), len(targets_dfmap), len(sources_dfsource), len(targets_dftarget))

print(len(sources_dfmap.difference(sources_dfsource)), len(targets_dfmap.difference(targets_dftarget)))

213 7


In [12]:
df_mars.insert(0, 'Division', 'Manufacturing')
df_mars.insert(1, 'Industry', 'Sugar and Confectionary Products')

df_mars.head(3)

,Division,Industry,Source Account #,Source Level 1,Source Level 2,Source Level 3,Source Level 4,Source Level 5,Source Level 6,Source Level 7,Source Level 8,Source Level 9,Source Level 10,Source Account Description,Target Account #,Target Level 1,Target Level 2,Target Level 3,Target Level 4,Target Account Description
0,Manufacturing,Sugar and Confectionary Products,110000,M003 Mars Financial Statement V2 (Full Trial B...,Profit and Loss,Operating Profit,6. Total Net Sales,3. Net Sales - third party domestic,Gross Sales - Domestic,NaN,NaN,NaN,NaN,Sales at List Price (Auto),400000,Revenue & Trade Expenditures,Revenue - 3rd Party,Revenue - Products - 3rd Party,Revenue - Products - 3rd Party Domestic,Gross Sales Products - 3rd Party Domestic - Auto
1,Manufacturing,Sugar and Confectionary Products,110005,M003 Mars Financial Statement V2 (Full Trial B...,Profit and Loss,Operating Profit,6. Total Net Sales,3. Net Sales - third party domestic,Gross Sales - Domestic,NaN,NaN,NaN,NaN,Sales at List Price - BIFG (Auto),400000,Revenue & Trade Expenditures,Revenue - 3rd Party,Revenue - Products - 3rd Party,Revenue - Products - 3rd Party Domestic,Gross Sales Products - 3rd Party Domestic - Auto
2,Manufacturing,Sugar and Confectionary Products,110010,M003 Mars Financial Statement V2 (Full Trial B...,Profit and Loss,Operating Profit,6. Total Net Sales,3. Net Sales - third party domestic,Gross Sales - Domestic,NaN,NaN,NaN,NaN,Sales at List Price Adjustment - Manual,400000,Revenue & Trade Expenditures,Revenue - 3rd Party,Revenue - Products - 3rd Party,Revenue - Products - 3rd Party Domestic,Gross Sales Products - 3rd Party Domestic - Auto


In [13]:
len(df_mars)

4379

In [14]:
df_mars = df_mars.sample(frac=1)
df_mars.to_csv(write_path+'CompanyB.csv', index=False)

## Company: Peleton

In [15]:
file = 'Peleton/GL_Mapping1.20.xlsx'
df_map = pd.read_excel(open(path+file, 'rb'), sheet_name="Sheet1", dtype=str, header=0)
df_map = df_map[df_map.columns[[0, 3]]]

file = 'Peleton/GL_SourceHierarchy1.20.xlsx'
df_source = pd.read_excel(open(path+file, 'rb'), sheet_name="Sheet1", dtype=str, header=0).query(f'K2 == "BS"')
df_source = df_source[df_source.columns[[2, 3, 4]]]

file = 'Peleton/GL_TargetHierarchy1.22.xlsx'
df_target = pd.read_excel(open(path+file, 'rb'), sheet_name="Sheet1", dtype=str, header=0).query(f'K2 == "BS"')
df_target = df_target[df_target.columns[[2, 3, 4]]]

df_map.head(3)

,S1,T1
0,10000,100120
1,10001,100080
2,10004,100100


In [16]:
df_pel = pd.DataFrame()
sources_not_found = []

for i in range(len(df_map)):
    #skip entries with empty sources/targets
    if any(df_map.iloc[i,:].isna()): continue
    #extract source and target numbers from mapping df
    source = df_map.loc[i, 'S1']
    target = df_map.loc[i, 'T1']

    if source not in df_source['account'].values:
        sources_not_found.append(source)
        continue
    else:
        #extract description of source number from source sheet 
        source_description = df_source.where(df_source['account'] == source).dropna(how='all')['Descriptions'].item()
        #extract source hierarchy label
        source_level = df_source.where(df_source['account'] == source).dropna(how='all')['K3'].item()

    if target not in df_target['account'].values:
        #print(f'Error: Target {target} from mapping sheet not found in target sheet')
        #continue
        target_description = 'NO MAPPING'
        target_level = 'NO MAPPING'
    else:
        find_target = df_target.where(df_target['account'] == target).dropna(how='all')
        if len(find_target) > 1:
            print(f'Error: target {target} maps to more than one account description')
            continue
        else:
            #extract description of target number from target sheet
            target_description = find_target['Descriptions'].item()
            #extract target hierarchy label
            target_level = find_target['K3'].item()

        
    df_pel = df_pel.append({'Source Account #': source, 'Source Level 1': source_level, 'Source Account Description': source_description,
                            'Target Account #': target, 'Target Level 1': target_level, 'Target Account Description': target_description,
                            }, ignore_index=True)


df_pel.insert(0, 'Division', 'Manufacturing')
df_pel.insert(1, 'Industry', 'Motorcycles, Bicycles, and Parts')

df_pel.head(3)

,Division,Industry,Source Account #,Source Level 1,Source Account Description,Target Account #,Target Level 1,Target Account Description
0,Manufacturing,"Motorcycles, Bicycles, and Parts",10000,Cash,Cash and Cash Equivalents,100120,Cash and Cash Equivalents,Cash (Parent: Legacy)
1,Manufacturing,"Motorcycles, Bicycles, and Parts",10001,Cash,FR Bank Operating - 7965,100080,Cash and Cash Equivalents,First Republic Bank Operating: 7965: Main
2,Manufacturing,"Motorcycles, Bicycles, and Parts",10004,Cash,FR Bank - Payroll - 2978,100100,Cash and Cash Equivalents,FR Bank: Payroll: 2978


In [17]:
len(df_pel)

447

In [18]:
len(sources_not_found)

484

In [19]:
df_pel = df_pel.sample(frac=1)
df_pel.to_csv(write_path+'CompanyC.csv', index=False)

## Company: Republic National Dist

In [20]:
file = 'Republic National Dist/Future State Mapping Consolidated Deliverable.xlsx'
df = pd.read_excel(open(path+file, 'rb'), sheet_name="Account", dtype=str, header=3)
df.head(3)

,Unnamed: 0,MasterMap,Company,AS400,Young's Oracle EBS Code,Young's Oracle EBS Description,RNDC Oracle EBS Account Code,RNDC Oracle EBS Sub-Account,RNDC Oracle EBS Account Description,Account Number Range,Account Type,General Account Number Range,General Account Classification,Specific Account Number Range,Specific Account Classification,Account Code,Account Name,Change Log/Comments
0,NaN,1000000000,AS400,1000000000,110100,MAIN CONCENTRATION,10020,**,NaN,100000-199999,Assets,100000-109999,Cash,101100-104999,Cash,101101,Cash - Master Cash,NaN
1,NaN,1000100000,AS400,1000100000,110100,MAIN CONCENTRATION,10020,**,NaN,100000-199999,Assets,100000-109999,Cash,101100-104999,Cash,101101,Cash - Master Cash,NaN
2,NaN,1000990000,AS400,1000990000,110100,MAIN CONCENTRATION,10020,**,NaN,100000-199999,Assets,100000-109999,Cash,101100-104999,Cash,101101,Cash - Master Cash,NaN


In [21]:
company = "Young's"
df_youngs = df.query(f'Company == "{company}"').dropna(how='all')

data_youngs = { 'Source Account #': df_youngs["Young's Oracle EBS Code"],
                'Source Account Description': df_youngs["Young's Oracle EBS Description"],
                'Target Account #': df_youngs['Account Code'],
                #'T L1': df_youngs['Account Type'],
                #'T L2': df_youngs['General Account Classification'],
                #'T L3': df_youngs['Specific Account Classification'],
                'Target Account Description': df_youngs['Account Name']}

df_rnd = pd.DataFrame(data_youngs)

df_rnd.insert(0, 'Division', 'Wholesale Trade')
df_rnd.insert(1, 'Industry', 'Groceries and Related Products')

df_rnd.head(3)

,Division,Industry,Source Account #,Source Account Description,Target Account #,Target Account Description
1182,Wholesale Trade,Groceries and Related Products,110100,MAIN CONCENTRATION,101101,Cash - Master Cash
1183,Wholesale Trade,Groceries and Related Products,110105,CENTRAL DEPOSITORY,101102,Cash - Depository
1184,Wholesale Trade,Groceries and Related Products,110106,B OF A LOCKBOX,101107,Cash - Lockbox


In [22]:
df_rnd = df_rnd.sample(frac=1)
df_rnd.to_csv(write_path+'CompanyD.csv', index=False)

## Company: JNJ

In [171]:
file = 'JNJ/JNJ GL mapping.xlsx'
df = pd.read_excel(open(path+file, 'rb'), sheet_name="GL Mapping", dtype=str, header=0).dropna(how='all').query('`Source Account Type` == "BS"')
df.head(3)

,Source System,Source Chart of Accounts,Source GL Account,Source GL Description,Source Account Type,Source Account Group,Target GL Account,Target Account Description Long,Target Account Description Short,Target Account Group,Target Account Type,Target Group Account
0,BTB NA,JNJG,111110001,Bank 1 - Primary Bank Account,BS,CASH,111110001,Bank 1 - Primary Bank Account,Bank1 - Primary Bank,CASH,BS,111110
1,BTB NA,JNJG,111110002,Bank 1 - GL Bank Sub Account,BS,CASH,111110002,Bank 1 - BS Bank Sub Account,Bank1-GL BankSubAcct,CASH,BS,111110
2,BTB NA,JNJG,111110003,Bank 1 - AR Bank Sub Account,BS,CASH,111110003,Bank 1 - AR Bank Sub Account,Bank1-AR BankSubAcct,CASH,BS,111110


In [172]:
data = {'Source Account #': df["Source GL Account"],
        #'S L1': np.nan,
        'Source Account Description': df["Source GL Description"],
        'Target Account #': df['Target GL Account'],
        #'T L1': np.nan,
        'Target Account Description': df['Target Account Description Short']}

df_jnj = pd.DataFrame(data)

df_jnj.insert(0, 'Division', 'Manufacturing')
df_jnj.insert(1, 'Industry', 'Drugs')

df_jnj.head(3)

,Division,Industry,Source Account #,Source Account Description,Target Account #,Target Account Description
0,Manufacturing,Drugs,111110001,Bank 1 - Primary Bank Account,111110001,Bank1 - Primary Bank
1,Manufacturing,Drugs,111110002,Bank 1 - GL Bank Sub Account,111110002,Bank1-GL BankSubAcct
2,Manufacturing,Drugs,111110003,Bank 1 - AR Bank Sub Account,111110003,Bank1-AR BankSubAcct


In [164]:
len(df_jnj)

7989

In [165]:
df_jnj = df_jnj.sample(frac=1)
df_jnj.to_csv(write_path+'CompanyE.csv', index=False) #, encoding='utf-16')

## Company: The Home Depot

In [27]:
file = 'The Home Depot/THD CoA Mapping.xlsx'
df_map = pd.read_excel(open(path+file, 'rb'), sheet_name="Sheet1", dtype=str, header=0)
df_map = df_map[df_map.columns[[2, 5]]]
df_map = df_map.drop_duplicates().reset_index(drop=True)

file = 'The Home Depot/ECC Account List.XLSX'
df_source = pd.read_excel(open(path+file, 'rb'), sheet_name="Sheet1", dtype=str, header=0)
df_source = df_source.query(f'`Language Key` == "EN"')
df_source = df_source[df_source.columns[[2, 4]]]

file = 'The Home Depot/S4 Account List.XLSX'
df_target = pd.read_excel(open(path+file, 'rb'), sheet_name="Sheet1", dtype=str, header=0)
df_target = df_target.query(f'`Language Key` == "EN"')
df_target = df_target[df_target.columns[[2, 4]]]

df_map.head(3)

,Source: G/L Account,Target: G/L Account
0,100010,1000010
1,100011,1000011
2,100015,1000015


In [28]:
len(df_map)

3060

In [29]:
df_thd = pd.DataFrame()

for i in range(len(df_map)):
    source = df_map.loc[i, 'Source: G/L Account']
    target = df_map.loc[i, 'Target: G/L Account']

    source_row = df_source.where(df_source['G/L Account'] == source).dropna(how='all')
    if len(source_row) == 0:
        print(f'Source account {source} from mapping sheet not found in source sheet')
        continue
    source_description = source_row['G/L Acct Long Text'].item()


    target_row = df_target.where(df_target['G/L Account'] == target).dropna(how='all')
    if len(target_row) == 0:
        #print(f'Target account {target} from mapping sheet not found in target sheet')
        #continue
        target_description = 'NO MAPPING'
    else:
        target_description = target_row['G/L Acct Long Text'].item()

    df_thd = df_thd.append({'Source Account #': source, 'Source Account Description': source_description,
                            'Target Account #': target, 'Target Account Description': target_description,
                            }, ignore_index=True)

df_thd.insert(0, 'Division', 'Retail Trade')
df_thd.insert(1, 'Industry', 'Hardware Stores')

df_thd.head(3)

,Division,Industry,Source Account #,Source Account Description,Target Account #,Target Account Description
0,Retail Trade,Hardware Stores,100010,Petty Cash,1000010,Petty Cash
1,Retail Trade,Hardware Stores,100011,PETTY CASH US DLLS,1000011,Petty Cash US Dollars
2,Retail Trade,Hardware Stores,100015,COD Change Orders,1000015,COD Change Orders


In [30]:
len(df_thd)

3060

In [31]:
df_thd = df_thd.sample(frac=1)
df_thd.to_csv(write_path+'CompanyF.csv', index=False)

## Company: Bausch Health

In [32]:
file = 'Bausch Health/4) Bausch Local to Global mapping BS 121118.xlsx'
df = pd.read_excel(open(path+file, 'rb'), sheet_name="Current COA Conso", dtype=str, header=0).dropna(how='all')
df = df[df.columns[[2, 3, 12, 13]]]
df.head(3)

,Source Acc Clean,Source Acc Description,S/4 Acct,S/4 Acct Description
0,101000,CASH IN BANK-CITI (USD),10000000,Cash
1,101000,CASH IN BANK-CITI (MYR),10000000,Cash
2,101000,CASH IN BANK-MBB222PJMYR,10000000,Cash


In [33]:
data = {'Source Account #': df["Source Acc Clean"],
        #'S L1': np.nan,
        'Source Account Description': df["Source Acc Description"],
        'Target Account #': df['S/4 Acct'],
        #'T L1': np.nan,
        'Target Account Description': df['S/4 Acct Description']}

df_bh = pd.DataFrame(data)

df_bh.insert(0, 'Division', 'Manufacturing')
df_bh.insert(1, 'Industry', 'Drugs')

df_bh

,Division,Industry,Source Account #,Source Account Description,Target Account #,Target Account Description
0,Manufacturing,Drugs,101000,CASH IN BANK-CITI (USD),10000000,Cash
1,Manufacturing,Drugs,101000,CASH IN BANK-CITI (MYR),10000000,Cash
2,Manufacturing,Drugs,101000,CASH IN BANK-MBB222PJMYR,10000000,Cash
3,Manufacturing,Drugs,101000,CASH IN BANK-CITI(USD)SU,10000000,Cash
4,Manufacturing,Drugs,101000,CASH IN BANK-CITI(SGD)SU,10000000,Cash
...,...,...,...,...,...,...
1675,Manufacturing,Drugs,103890,Other Inventory Reserves ICMU,12900250,Allow. for Finished Goods Inventory Obsol. - ICMU
1676,Manufacturing,Drugs,103890,Other Inventory Reserves ICMU,12900250,Allow. for Finished Goods Inventory Obsol. - ICMU
1677,Manufacturing,Drugs,103890,Other Inventory Reserves ICMU,12900250,Allow. for Finished Goods Inventory Obsol. - ICMU
1678,Manufacturing,Drugs,103844,Inventory Samples ICMU,13210050,Trade samples - non-saleable - ICMU


In [34]:
len(df_bh)

1680

In [35]:
df_bh = df_bh.sample(frac=1)
df_bh.to_csv(write_path+'CompanyG.csv', index=False)

## Company: AT&T

In [36]:
file = 'AT&T COA Mapping/HBO Account Mapping.xlsx'
df = pd.read_excel(open(path+file, 'rb'), sheet_name="BS Account Mapping", dtype=str, header=4).dropna(how='all')
df.head(3)

,HFM Account,HFM Account Description,Account Type,TC3,Global GL Account,Global GL Account Description,Hyperion Account,Hyperion Description
0,1020,1020 - Cash Disbursements - Mellon A/P Disburs...,Assets,ENDING,1130.1,CASH,A10000,Cash
1,1022,1022 - Cash Disbursements - Chase - HBO Payroll,Assets,ENDING,1130.1,CASH,A10000,Cash
2,1026,1026 - Cash Disbursements - Chase Controlled D...,Assets,ENDING,1130.1,CASH,A10000,Cash


In [37]:
data = {'Source Account #': df["HFM Account"],
        'Source Account Description': df["HFM Account Description"],
        'Target Account #': df["Global GL Account"],
        'Target Account Description': df["Global GL Account Description"]}

df_att = pd.DataFrame(data).dropna()

df_att.insert(0, 'Division', 'Transportation, Communications, Electric, Gas, And Sanitary Services')
df_att.insert(1, 'Industry', 'Telephone Communications')

df_att

,Division,Industry,Source Account #,Source Account Description,Target Account #,Target Account Description
0,"Transportation, Communications, Electric, Gas,...",Telephone Communications,1020,1020 - Cash Disbursements - Mellon A/P Disburs...,1130.1,CASH
1,"Transportation, Communications, Electric, Gas,...",Telephone Communications,1022,1022 - Cash Disbursements - Chase - HBO Payroll,1130.1,CASH
2,"Transportation, Communications, Electric, Gas,...",Telephone Communications,1026,1026 - Cash Disbursements - Chase Controlled D...,1130.1,CASH
3,"Transportation, Communications, Electric, Gas,...",Telephone Communications,1000,1000 - Cash Receipts - Chase Manhattan - NY Lo...,1130.1,CASH
4,"Transportation, Communications, Electric, Gas,...",Telephone Communications,1001,1001 - Cash Receipts - Chase Manhattan - TLF/HBOe,1130.1,CASH
...,...,...,...,...,...,...
421,"Transportation, Communications, Electric, Gas,...",Telephone Communications,2606IC,2606IC - LT Programming Lic Fees - Assets - In...,1422.11,"N/C Acquired Programming Costs, less amort."
422,"Transportation, Communications, Electric, Gas,...",Telephone Communications,2620IC,2620IC - LT Prepaid Programming-Original - Int...,1422.11,"N/C Acquired Programming Costs, less amort."
423,"Transportation, Communications, Electric, Gas,...",Telephone Communications,2129,"CIP - Equip, F_F and LHI 1",2003.1099C,Tele PUC - Other Conversion
424,"Transportation, Communications, Electric, Gas,...",Telephone Communications,2008,Investment in Subs I/C - Minority Interest,1401.7A,Investment in Subs I/C - Minority Interest


In [38]:
df_att = df_att.sample(frac=1)
df_att.to_csv(write_path+'CompanyH.csv', index=False)

## Company: Energizer

In [39]:
file = "Energizer CFIN Implementation/Energizer SAP Legacy to CFIN GL Mapping.xlsx"
df = pd.read_excel(open(path+file, 'rb'), sheet_name="SAP Legacy to CFIN GL-Mapping", dtype=str, header=0).query(f'`Balance Sheet Account` == "X"').dropna(how='all')
df.head(3)

,System,Chart of Accounts,G/L Account,Balance Sheet Account,Group account number,Account Group,Short Text,Long Text,CFIN COA,CFIN Account (XXXXXX),CFIN Account Type,CFIN Group Account Number,Group Account Description,CFIN Account Group,CFIN account short text,CFIN account long text,Comments
0,Prime,GCOA,227005,X,A0305108,ZCUL,Acc Bonus-Prov. Fund,Acc Bonus-Accr Provident Fund on Bonus,GCOA,220100,X,2201,Accrued Incentive Comp & Bonuses,ZCUL,Acc Bonus-Prov. Fund,Acc Bonus-Accr Provident Fund on Bonus,NaN
1,Prime,GCOA,227001,X,A0305108,ZCUL,Acc Bonus-FieldSales,Acc Bonus-Field Sales Bonus,GCOA,220101,X,2201,Accrued Incentive Comp & Bonuses,ZCUL,Acc Bonus-FieldSales,Acc Bonus-Field Sales Bonus,NaN
2,Prime,GCOA,227004,X,A0305108,ZCUL,Acc Bonus-HQ Market,Acc Bonus-HQ Market Bonus,GCOA,220102,X,2201,Accrued Incentive Comp & Bonuses,ZCUL,Acc Bonus-HQ Market,Acc Bonus-HQ Market Bonus,NaN


In [40]:
data = {'Source Account #': df["G/L Account"],
        'Source Account Description': df["Long Text"],
        'Target Account #': df["CFIN Account (XXXXXX)"],
        'Target Account Description': df["CFIN account long text"]}

df_ener = pd.DataFrame(data).dropna() #drop rows with blanks for source/target

df_ener.insert(0, 'Division', 'Manufacturing')
df_ener.insert(1, 'Industry', 'Miscellaneous Electrical')

df_ener


,Division,Industry,Source Account #,Source Account Description,Target Account #,Target Account Description
0,Manufacturing,Miscellaneous Electrical,227005,Acc Bonus-Accr Provident Fund on Bonus,220100,Acc Bonus-Accr Provident Fund on Bonus
1,Manufacturing,Miscellaneous Electrical,227001,Acc Bonus-Field Sales Bonus,220101,Acc Bonus-Field Sales Bonus
2,Manufacturing,Miscellaneous Electrical,227004,Acc Bonus-HQ Market Bonus,220102,Acc Bonus-HQ Market Bonus
3,Manufacturing,Miscellaneous Electrical,227000,Acc Bonus-Incentive Comp,220103,Acc Bonus-Incentive Comp
4,Manufacturing,Miscellaneous Electrical,227006,Acc Bonus-Profit Sharing,220104,Acc Bonus-Profit Sharing
...,...,...,...,...,...,...
3878,Manufacturing,Miscellaneous Electrical,233077,All Other Accruals - Turnover W/H Tucuman,229075,All Other Accruals - Turnover W/H Tucuman
3879,Manufacturing,Miscellaneous Electrical,233078,All Other Accruals - Turnover tax receivable,229076,All Other Accruals - Turnover tax receivable
3880,Manufacturing,Miscellaneous Electrical,245003,Income Taxes Payable - Self W/H,206004,Income Taxes Payable - Self W/H
3890,Manufacturing,Miscellaneous Electrical,U100512,Cash - In Transit,104219,Cash - In Transit


In [41]:
df_ener = df_ener.sample(frac=1)
df_ener.to_csv(write_path+'CompanyI.csv', index=False)

In [42]:
len(df_ener)

1677

## Company: WESCO

In [43]:
file = "WESCO COA Mapping/Wesco Account Mapping.xlsx"
df = pd.read_excel(open(path+file, 'rb'), sheet_name="GL Account", dtype=str, header=0).dropna(how='all')
df.head(3)

,Source Account,Source Account Description,Source Account Parent,Source Account Parent Description,Target Account (L1),Target Account Description (L1),Target Child Account Number (L2),Target Child Account Description (L2),Target Account Parent (L3),Target Parent Description (L3),General Account Number (L4),Account Classification Description (L4),Target Account Type Number (L5),Target Account Type (L5)
0,30201,Treasury stock - TREASURY STOCK,30200,TREASURY STOCK,304001,TREASURY STOCK - TREASURY STOCK,NaN,NaN,304000,TREASURY STOCK,300000,STOCKHOLDER'S EQUITY,300000,EQUITY
1,70101,TRANSPORTATION OUT - TRUCK RENTAL,70100,TRANSPORTATION OUT,622001,TRANSPORTATION OUT - TRUCK RENTAL,NaN,NaN,622000,TRANSPORTATION OUT,620000,"SELLING, GENERAL, AND ADMINISTRATIVE",600000,OPERATING EXPENSES
2,70102,TRANSPORTATION OUT - TRUCK OPERATION EXPENSE,70100,TRANSPORTATION OUT,622002,TRANSPORTATION OUT - TRUCK OPERATION EXPENSE,NaN,NaN,622000,TRANSPORTATION OUT,620000,"SELLING, GENERAL, AND ADMINISTRATIVE",600000,OPERATING EXPENSES


In [44]:
data = {'Source Account #': df["Source Account"],
        'Source Account Description': df["Source Account Description"],
        'Target Account #': df["Target Account (L1)"],
        'Target Account Description': df["Target Account Description (L1)"]}

df_wesco = pd.DataFrame(data).dropna() #drop rows with blanks for source/target

df_wesco.insert(0, 'Division', 'Retail Trade')
df_wesco.insert(1, 'Industry', 'Grocery Stores')

df_wesco

,Division,Industry,Source Account #,Source Account Description,Target Account #,Target Account Description
0,Retail Trade,Grocery Stores,30201,Treasury stock - TREASURY STOCK,304001,TREASURY STOCK - TREASURY STOCK
1,Retail Trade,Grocery Stores,70101,TRANSPORTATION OUT - TRUCK RENTAL,622001,TRANSPORTATION OUT - TRUCK RENTAL
2,Retail Trade,Grocery Stores,70102,TRANSPORTATION OUT - TRUCK OPERATION EXPENSE,622002,TRANSPORTATION OUT - TRUCK OPERATION EXPENSE
3,Retail Trade,Grocery Stores,70104,TRANSPORTATION OUT - TRUCK DRIVER SALARY RECLASS,622004,TRANSPORTATION OUT - TRUCK DRIVER SALARY RECLASS
4,Retail Trade,Grocery Stores,70105,TRANSPORTATION OUT - TRANSPORTATION OUT,622006,TRANSPORTATION OUT - TO CUSTOMER
...,...,...,...,...,...,...
902,Retail Trade,Grocery Stores,20030,ACCOUNTS PAYABLE - CUSTOMER VOLUME REBATES,211017,ACCOUNTS PAYABLE - CUSTOMER VOLUME REBATES
903,Retail Trade,Grocery Stores,20008,ACCOUNTS PAYABLE - CORPORATE DIRECT SHIP ACCRUAL,211008,ACCOUNTS PAYABLE - CORPORATE DIRECT SHIP ACCRUAL
904,Retail Trade,Grocery Stores,20009,ACCOUNTS PAYABLE - BRANCH DIRECT SHIP ACCRUAL,211009,ACCOUNTS PAYABLE - BRANCH DIRECT SHIP ACCRUAL
905,Retail Trade,Grocery Stores,20096,ACCOUNTS PAYABLE - AP Zero Payment Process Cle...,211031,ACCOUNTS PAYABLE - AP ZERO PAYMENT PROCESS CLE...


In [45]:
df_wesco = df_wesco.sample(frac=1)
df_wesco.to_csv(write_path+'CompanyJ.csv', index=False)

In [46]:
len(df_wesco)

907

## Company: Elanco Animal Health

In [47]:
df_list_full = [df_att, df_pel, df_rnd, df_wesco, df_fan, df_ener, df_bh, df_thd, df_mars, df_jnj]
sum = 0
for df in df_list_full:
    sum += len(df)

for df in df_list_full:
    print(f'{((len(df)/sum)*100):.2f} %')

1.89 %
1.98 %
2.61 %
4.03 %
6.10 %
7.44 %
7.46 %
13.58 %
19.44 %
35.46 %


In [48]:
df_sap_list = [df_pel, df_ener, df_bh, df_thd, df_mars, df_jnj]
sum = 0
for df in df_sap_list:
    sum += len(df)

for df in df_sap_list:
    print(f'{((len(df)/sum)*100):.2f} %')

2.32 %
8.72 %
8.74 %
15.91 %
22.77 %
41.54 %


In [173]:
def preprocess_text(text):

    text = str(text)
    text = re.sub('-', ' ', text)
    text = re.sub('–', ' ', text)
    text = re.sub("'", '', text)
    text = text.split(' ')
   
    words = []

    for word in text:

        #handle text with forward slash
        fs_split = word.split('/')
        if len(fs_split) > 1:
            if(list(filter(lambda word: len(word) < 3, fs_split))): word = "".join(fs_split)
            else: word = " ".join(fs_split)

        #handle text with ampersand 
        amp_split = word.split('&')
        if len(amp_split) > 1:
            if(list(filter(lambda word: len(word) < 3, amp_split))): word = "".join(amp_split)
            else: word = " ".join(amp_split)

        #handle text with period abbreviations
        per_split = word.split('.')
        if len(per_split) > 1:
            if(list(filter(lambda word: len(word) < 3, per_split))): word = "".join(per_split)
            else: word = " ".join(per_split)
        
        #handle single strings with words separated by capitals
        chars = [char for char in word]

        idx_slices = [0]
        for i in range(len(chars)):
            if i == 0: continue
            #want to keep consecutive letters together
            if chars[i-1] in string.ascii_uppercase: continue
            if chars[i] in string.ascii_uppercase: idx_slices.append(i)

        word = " ".join([word[i:j] for i,j in zip(idx_slices, idx_slices[1:]+[None])])
        words.append(word)
    
    text = " ".join(words).lower().strip()
    text = "".join([char for char in text if char not in string.punctuation+string.digits])
    text = re.sub('\\s+', ' ', text).strip()

    return text

In [174]:
text = "Maint.softw.th.parti"
#text = 'U.S. ResChrgbks/CustRebEB'

preprocess_text(text)

'maintsoftwthparti'

In [179]:
df = df_mars
type = 'Source'

original = df[f'{type} Account Description'].to_list()
processed = df[f'{type} Account Description'].apply(preprocess_text).to_list()

data = {'Original': original,
        'Processed': processed}

df_preproc = pd.DataFrame(data)

df_preproc.to_csv("c:\\Users\\devquinn\\OneDrive - Deloitte (O365D)\\Desktop\\preproc.csv", index=False, mode='w+')

In [149]:
file = 'JNJ/JNJ GL mapping.xlsx'
df = pd.read_excel(open(path+file, 'rb'), sheet_name="GL Mapping", dtype=str, header=0).dropna(how='all')
df.head(3)

,Source System,Source Chart of Accounts,Source GL Account,Source GL Description,Source Account Type,Source Account Group,Target GL Account,Target Account Description Long,Target Account Description Short,Target Account Group,Target Account Type,Target Group Account
0,BTB NA,JNJG,111110001,Bank 1 - Primary Bank Account,BS,CASH,111110001,Bank 1 - Primary Bank Account,Bank1 - Primary Bank,CASH,BS,111110
1,BTB NA,JNJG,111110002,Bank 1 - GL Bank Sub Account,BS,CASH,111110002,Bank 1 - BS Bank Sub Account,Bank1-GL BankSubAcct,CASH,BS,111110
2,BTB NA,JNJG,111110003,Bank 1 - AR Bank Sub Account,BS,CASH,111110003,Bank 1 - AR Bank Sub Account,Bank1-AR BankSubAcct,CASH,BS,111110


In [150]:
df.iloc[3955, :]['Source GL Description']

'Suspense Accounts Payable USD- Revaluatión'

In [151]:
data = {'Source Account #': df["Source GL Account"],
        #'S L1': np.nan,
        'Source Account Description': df["Source GL Description"],
        'Target Account #': df['Target GL Account'],
        #'T L1': np.nan,
        'Target Account Description': df['Target Account Description Short']}

df = pd.DataFrame(data)

df.insert(0, 'Division', 'Manufacturing')
df.insert(1, 'Industry', 'Drugs')

In [152]:
df.iloc[3955, :]['Source Account Description']

'Suspense Accounts Payable USD- Revaluatión'

In [154]:
df.to_csv("c:\\Users\\devquinn\\OneDrive - Deloitte (O365D)\\Desktop\\experiment.csv", index=False, mode='w+', encoding='utf-16')

In [147]:
df = df.sample(frac=1)
df.iloc[3955, :]['Source Account Description']

'RESERVE FOR VOLUME DISCOUNT TO CUSTOMERS A L'

In [ ]:
df_jnj.to_csv(write_path+'CompanyE.csv', index=False, encoding='utf-16')

In [52]:
from collections import Counter

def find_most_common_words(df, top_n=5):
    words = []
    tar_accts = df['Target Account Description'].apply(preprocess_text).to_list()
    src_accts = df['Source Account Description'].apply(preprocess_text).to_list()
    src_accts.extend(tar_accts)
    for a in src_accts:
        a_split = a.split(' ')
        for w in a_split:
            if (len(w) > 1) and (len(w) < 4) :
                words.append(w)

    if top_n == 'all':
        return Counter(words)
    else:
        return Counter(words).most_common(top_n)

word_count = {}

for df in df_list_full:
    top_words = find_most_common_words(df, top_n=10)
    for i in range(len(top_words)):
        word = top_words[i][0]
        count = top_words[i][1]
        if word in word_count.keys():
            word_count[word] += count
        else:
            word_count[word] = count

sorted_word_count = dict(sorted((value, key) for (key,value) in word_count.items()))
total_count = np.sum(list(sorted_word_count.keys()))

word_percentages = {}

for word in word_count.keys():
    count = word_count[word]
    percent = round((count/total_count)*100, 3)
    word_percentages[word] = percent

sorted_word_percentages = dict(sorted((value, key) for (key,value) in word_percentages.items()))
sorted_word_percentages

{0.054: 'DE',
 0.059: 'UK',
 0.074: 'IN',
 0.098: 'Of',
 0.108: 'RM',
 0.118: 'Dep',
 0.132: 'Acc',
 0.147: 'To',
 0.157: 'Uk',
 0.162: 'EE',
 0.201: 'TWX',
 0.211: 'Loc',
 0.235: 'US',
 0.245: 'END',
 0.255: 'ERP',
 0.28: 'Use',
 0.353: 'EU',
 0.358: 'GTN',
 0.378: 'Ar',
 0.388: 'NC',
 0.402: 'EUR',
 0.412: 'on',
 0.491: 'PNC',
 0.564: 'ST',
 0.638: 'ACC',
 0.751: 'Emp',
 0.755: 'HD',
 0.814: 'OE',
 0.854: 'Std',
 0.893: 'Ltd',
 1.104: 'TAX',
 1.138: 'ACH',
 1.172: 'LT',
 1.275: 'CAD',
 1.329: 'rd',
 1.501: 'PPE',
 2.276: 'AP',
 2.389: 'AR',
 2.428: 'Man',
 2.654: 'Trd',
 3.056: 'in',
 3.247: 'of',
 3.669: 'Adj',
 3.954: 'All',
 4.096: 'OIM',
 4.126: 'ECC',
 4.346: 'Oth',
 4.724: 'IC',
 4.729: 'Out',
 4.935: 'In',
 5.136: 'VAT',
 5.156: 'Rec',
 5.381: 'Pay',
 6.676: 'Non',
 8.913: 'Tax'}

In [53]:
common_abbreviations = []

for df in df_list_full:
    df_abbr = list(list(zip(*find_most_common_words(df, top_n=10)))[0])
    for a in df_abbr:
        if (a in common_abbreviations): continue
        else: common_abbreviations.append(a)

common_abbreviations = sorted(common_abbreviations, key=str.lower)

with open("c:\\Users\\devquinn\\OneDrive - Deloitte (O365D)\\Desktop\\common_abbreviations.txt", 'w+') as abbr_file:
    abbr_file.writelines("%s\n" % a for a in common_abbreviations)


In [55]:
abbreviations = []

for df in df_list_full:

    target_accounts = df['Target Account Description'].apply(preprocess_text).tolist()
    for ta in target_accounts:
        words = ta.split(' ')
        for word in words:
            if len(word) < 4:
                if word in abbreviations: continue
                abbreviations.append(word)

    source_accounts = df['Source Account Description'].apply(preprocess_text).tolist()
    for sa in source_accounts:
        words = sa.split(' ')
        for word in words:
            if len(word) < 4:
                if word in abbreviations: continue
                abbreviations.append(word)

len(abbreviations)


1597

In [56]:
abbreviations = sorted(abbreviations, key=str.lower)

In [57]:
with open("c:\\Users\\devquinn\\OneDrive - Deloitte (O365D)\\Desktop\\abbreviations.txt", 'w+') as abbreviations_file:
    abbreviations_file.writelines("%s\n" % a for a in abbreviations)

## Combine Datasets

In [110]:
#System / Source ERP
df_sys_oracle = pd.concat([df_att, df_rnd, df_wesco])
df_sys_oracle.insert(0, 'System', 'Oracle')
df_sys_oracle.to_csv(write_path+'Oracle.csv', index=False)

df_sys_sap = pd.concat([df_bh, df_ener, df_jnj, df_mars, df_pel, df_thd])
df_sys_sap.insert(0, 'System', 'SAP')
df_sys_sap.to_csv(write_path+'SAP.csv', index=False)

df_sys_msdyn = pd.concat([df_fan])
df_sys_msdyn.insert(0, 'System', 'MSDynamics')
df_sys_msdyn.to_csv(write_path+'MSDynamics.csv', index=False)

#Industry
df_ind_tmt = pd.concat([df_att]).rename(columns={'Industry': 'Sector'})
df_ind_tmt.insert(0, 'Industry', 'TMT')
df_ind_tmt.to_csv(write_path+'TMT.csv', index=False)

df_ind_lshc = pd.concat([df_bh, df_jnj]).rename(columns={'Industry': 'Sector'})
df_ind_lshc.insert(0, 'Industry', 'LSHC')
df_ind_lshc.to_csv(write_path+'LSHC.csv', index=False)

df_ind_consumercp = pd.concat([df_ener, df_fan, df_mars, df_pel, df_rnd, df_thd, df_wesco]).rename(columns={'Industry': 'Sector'})
df_ind_consumercp.insert(0, 'Industry', 'Consumer-CP')
df_ind_consumercp.to_csv(write_path+'Consumer-CP.csv', index=False)


In [113]:
write_path

'C:/Users/devquinn/OneDrive - Deloitte (O365D)/Desktop/atom-dc-coa/dc-atom-coa/project/data/'

In [112]:
df_systems = [df_sys_sap, df_sys_oracle, df_sys_msdyn]

df_systems[0]['Target Account Description']

1611                                    Paid in Capital
766              PPE - Accum. Depr. - Computer Software
621                                PPE - Demo equipment
827          PPE - Accum. Depr. - Machinery & Equipment
684                                PPE - Demo equipment
                             ...                       
1052                   CFIN ECC Blocked Acct - Cash OIM
560                    CFIN ECC Blocked Acct - Cash OIM
1134                       Wells Fargo Disbursement SSC
1364               CFIN ECC Blocked Acct - Cash Non OIM
2939    OPEX F&D - Inbound Freight DF Order Integration
Name: Target Account Description, Length: 19232, dtype: object

In [ ]:
from sklearn.model_selection import train_test_split
df_all = pd.concat([df_mars, df_fan, df_pel, df_rnd, df_jnj, df_thd, df_bh])
df_all.columns=['Division', 'Industry', 'Source Account #', 'Source Level 1', 'Source Level 2', 'Source Level 3', 'Source Level 4', 'Source Level 5', 'Source Level 6', 'Source Level 7', 'Source Level 8', 
                'Source Level 9', 'Source Level 10', 'Source Account Description', 'Target Account #', 'Target Level 1', 'Target Level 2', 'Target Level 3','Target Level 4', 'Target Account Description']

print(len(df_all))
df_all.head(3)


In [ ]:
#shuffle data
df_all = df_all.sample(frac=1)
write_path = 'C:/Users/devquinn/OneDrive - Deloitte (O365D)/Desktop/atom-dc-coa/dc-atom-coa/project/data/'
df_all.to_excel(write_path+'data.xlsx', index=False)
print(f'Data excel file written to {write_path}')


In [ ]:
data_train, data_test = train_test_split(df_all, test_size=0.15, shuffle=True, random_state=12)
write_path = 'C:/Users/devquinn/OneDrive - Deloitte (O365D)/Desktop/atom-dc-coa/dc-atom-coa/project/data/'
data_train.to_excel(write_path+'data_train.xlsx', index=False)
data_test.to_excel(write_path+'data_test.xlsx', index=False)
print(f'Train and test excel files written to {write_path}')